In [6]:
from db.connect import connect
con = connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.5 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44), 64-bit',)


In [10]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_sql_query('select * from "atendimentos"',con=con)
ultimo_registro = df[df["tipo_senha"] == "SP"].tail(1)
ultimo_registro
data_ultimo_registro = pd.to_datetime(ultimo_registro["data_emissao"]).values[0]
data_ultimo_registro

numpy.datetime64('2022-09-07T01:12:53.247825000')

In [19]:
from datetime import datetime, timedelta

Timestamp('2022-09-07 01:12:53.247825+0000', tz='UTC')

In [30]:
pd.Timestamp(data_ultimo_registro).replace(tzinfo=datetime.timezone.utc).astimezone(pytz.timezone('America/Recife'))

Timestamp('2022-09-06 22:12:53.247825-0300', tz='America/Recife')

In [26]:
import pytz

In [32]:
datetime.datetime.now(pytz.timezone('America/Recife'))

datetime.datetime(2022, 9, 6, 22, 40, 47, 614611, tzinfo=<DstTzInfo 'America/Recife' -03-1 day, 21:00:00 STD>)

In [12]:
pd.to_datetime(datetime.datetime.now(datetime.timezone.utc).replace(hour = 7, minute = 0, second = 0, microsecond = 0))

Timestamp('2022-09-07 07:00:00+0000', tz='UTC')

In [3]:
import requests
import random
import time
import tqdm

env = ['http://localhost:8000/', 'https://ticketprint.herokuapp.com/']

env = env[1]

opcoes = ['SG','SP','SE']

for i in tqdm.tqdm(range(0,10)):
    index = random.randint(0,2)
    url = env + "senha/" + opcoes[index]
    r = requests.get(url)
    time.sleep(index)

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


In [33]:
import requests
import random
import time
import tqdm

env = ['http://localhost:8000/', 'https://ticketprint.herokuapp.com/']

env = env[0]

for i in range(0,11):
    guiche = random.randint(1,5)
    url = env + "chamada/" + str(guiche)
    r = requests.get(url)
    print(r.json()) 
    time.sleep(2)

{'senha': 'SP003', 'data_atendimento': '06/09/2022, 22:51:03', 'guiche': '3'}
{'senha': 'SP004', 'data_atendimento': '06/09/2022, 22:51:09', 'guiche': '1'}
{'senha': 'Não há senhas a serem chamadas'}
{'senha': 'Não há senhas a serem chamadas'}
